# Deploying a MedNIST Classifier App with MONAI Deploy App SDK (Prebuilt Model)

This tutorial demos the process of packaging up a trained model using MONAI Deploy App SDK into an artifact which can be run as a local program performing inference, a workflow job doing the same, and a Docker containerized workflow execution.

In this tutorial, we will use a trained model and implement & package the inference application, executing the application locally.


## Clone the github project (the latest version of the main branch only)


In [1]:
!rm -rf source \
 && git clone --branch main --depth 1 https://github.com/Project-MONAI/monai-deploy-app-sdk.git source \
 && rm -rf source/.git

Cloning into 'source'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 287 (delta 61), reused 121 (delta 20), pack-reused 0
Receiving objects: 100% (287/287), 1.21 MiB | 10.01 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [2]:
!ls source/examples/apps/mednist_classifier_monaideploy/

mednist_classifier_monaideploy.py


## Install monai-deploy-app-sdk package

In [3]:
!pip install --upgrade monai-deploy-app-sdk

## Install necessary packages for the app

In [4]:
!pip install monai Pillow  # for MONAI transforms and Pillow
!python -c "import pydicom" || pip install -q "pydicom>=1.4.2"
!python -c "import highdicom" || pip install -q "highdicom>=0.18.2" # for the use of DICOM Writer operators

## Download/Extract mednist_classifier_data.zip from Google Drive

In [5]:
# Download mednist_classifier_data.zip
!pip install gdown 
!gdown "https://drive.google.com/uc?id=1yJ4P-xMNEfN6lIOq_u6x1eMAq1_MJu-E"

Downloading...
From (uriginal): https://drive.google.com/uc?id=1yJ4P-xMNEfN6lIOq_u6x1eMAq1_MJu-E
From (redirected): https://drive.google.com/uc?id=1yJ4P-xMNEfN6lIOq_u6x1eMAq1_MJu-E&confirm=t&uuid=64399e9b-2546-4500-9f28-b298c7c5c684
To: /home/mqin/src/monai-deploy-app-sdk/notebooks/tutorials/mednist_classifier_data.zip
100%|██████████████████████████████████████| 28.6M/28.6M [00:00<00:00, 64.6MB/s]


In [6]:
# After downloading mednist_classifier_data.zip from the web browser or using gdown,
!unzip -o "mednist_classifier_data.zip"

Archive:  mednist_classifier_data.zip
 extracting: classifier.zip          
 extracting: input/AbdomenCT_007000.jpeg  


## Package app (creating MAP Docker image)

This assumes that nvidia docker is installed in the local machine.

Please see https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html#docker to install nvidia-docker2.

Use `-l DEBUG` option to see progress.

In [7]:
!monai-deploy package "source/examples/apps/mednist_classifier_monaideploy/mednist_classifier_monaideploy.py" \
    --tag mednist_app:latest \
    --model classifier.zip

Building MONAI Application Package... -[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (1/2)                                                         
 => [internal] load build definition from dockerfile                       0.1s
 => => transferring dockerfile: 2.58kB                                     0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 1.11kB                                        0.0s
[?25\[+] Building 0.2s (4/19)                                                        
 => [internal] load build definition from dockerfile                       0.1s
 => => transferring dockerfile: 2.58kB                                     0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 1.11kB                                        0.0s
 => [internal] load metadata for nvcr.io/nvidia/pytorch:22.08-py3       

## Run the app with docker image and input file locally

In [8]:
!monai-deploy run mednist_app:latest "input" "output"

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "mednist_app:latest" is available...

Checking for MAP "mednist_app:latest" locally
"mednist_app:latest" found.

Reading MONAI App Package manifest...
Preparing to copy...Copying from container - 0BSuccessfully copied 2.05kB to /tmp/tmpvdf4hnad/app.json
Preparing to copy...Copying from container - 0BSuccessfully copied 2.05kB to /tmp/tmpvdf4hnad/pkg.json
--> Verifying if "nvidia-docker" is installed...

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 1, Operator ID: 5ca4fdc7-8dd0-4eb8-8550-9d92b820f2da)
Done performing execution of operator LoadPILOperator

Going to ini

In [9]:
!cat output/output.json

"AbdomenCT"

## Implementing and Packaging Application with MONAI Deploy App SDK

Based on the Torchscript model(`classifier.zip`), we will implement an application that process an input Jpeg image and write the prediction(classification) result as JSON file(`output.json`).

In our inference application, we will define two operators:

1. `LoadPILOperator` - Load a JPEG image from the input path and pass the loaded image object to the next operator.
    - This Operator does similar job with `LoadImage(image_only=True)` transform in *train_transforms*, but handles only one image.
    - **Input**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
2. `MedNISTClassifierOperator` - Pre-transform the given image by using MONAI's `Compose` class, feed to the Torchscript model (`classifier.zip`), and write the prediction into JSON file(`output.json`)
    - Pre-transforms consist of three transforms -- `AddChannel`, `ScaleIntensity`, and `EnsureType`.
    - **Input**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output**: a folder path that the prediction result(`output.json`) would be written ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))

The workflow of the application would look like this.

<img src="https://user-images.githubusercontent.com/1928522/133868503-46671f0a-7741-4f9d-aefa-83e95e9a5f84.png" alt="Workflow" style="width: 600px;margin-left:auto;margin-right:auto;"/>


### Setup imports

Let's import necessary classes/decorators and define `MEDNIST_CLASSES`.

In [10]:
import monai.deploy.core as md
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]

### Creating Operator classes

#### LoadPILOperator

In [11]:
@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """Load image from the given input (DataPath) and set numpy array to the output (Image)."""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image = PILImage.open(input_path)
        image = image.convert("L")  # convert to greyscale image
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # create Image domain object with a numpy array
        op_output.set(output_image)

#### MedNISTClassifierOperator

In [12]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # get a TorchScriptModel object

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # get the class name
        print(result)

        # Get output (folder) path and create the folder if not exists
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)

### Creating Application class

Our application class would look like below.

It defines `App` class inheriting `Application` class.

`LoadPILOperator` is connected to `MedNISTClassifierOperator` by using `self.add_flow()` in `compose()` method of `App`.

In [13]:
@md.resource(cpu=1, gpu=1, memory="1Gi")
@md.env(pip_packages=["pydicom >= 2.3.0", "highdicom>=0.18.2"])  # for the use of DICOM Writer operators
class App(Application):
    """Application class for the MedNIST classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)

### Executing app locally

We can execute the app in the Jupyter notebook. Before doing so, we also need to clean the output folder which was created by running the packaged containerizd app in the previous cell.

In [14]:
!rm -rf output

app = App()

In [15]:
app.run(input="input/AbdomenCT_007000.jpeg", output="output", model="classifier.zip")

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 409880, Operator ID: 06991530-662d-46eb-83f2-e8e1368c4bb1)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 409880, Operator ID: 1c4a4293-869f-4843-83f0-2c857c3bde96)


/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:111: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead with `channel_dim='no_channel'`.
  warn_deprecated(obj, msg, warning_category)
/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/monai/data/meta_tensor.py:116: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return torch.as_tensor(x, *args, **_kwargs).as_subcl

AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [16]:
!cat output/output.json

"AbdomenCT"

Once the application is verified inside Jupyter notebook, we can write the whole application as a file(`mednist_classifier_monaideploy.py`) by concatenating code above, then add the following lines:

```python
if __name__ == "__main__":
    App(do_run=True)
```

The above lines are needed to execute the application code by using `python` interpreter.

In [17]:
%%writefile mednist_classifier_monaideploy.py

# Copyright 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]


@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """Load image from the given input (DataPath) and set numpy array to the output (Image)."""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image = PILImage.open(input_path)
        image = image.convert("L")  # convert to greyscale image
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # create Image domain object with a numpy array
        op_output.set(output_image)


@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # get a TorchScriptModel object

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # get the class name
        print(result)

        # Get output (folder) path and create the folder if not exists
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)


@md.resource(cpu=1, gpu=1, memory="1Gi")
@md.env(pip_packages=["pydicom >= 2.3.0", "highdicom>=0.18.2"])  # for the use of DICOM Writer operators
class App(Application):
    """Application class for the MedNIST classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)


if __name__ == "__main__":
    App(do_run=True)

Overwriting mednist_classifier_monaideploy.py


In this time, let's execute the app in the command line.

In [18]:
!python "mednist_classifier_monaideploy.py" -i "input/AbdomenCT_007000.jpeg" -o output -m "classifier.zip"

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 410692, Operator ID: c6ecd716-26ef-4706-8cd4-11fc7f3179bb)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 410692, Operator ID: 7b972a34-7f9b-472e-ad4c-394526e9c20b)
/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:111: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead with `channel_dim='no_channel'`.
  warn_deprecated(obj, msg, warning_category)
/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/monai/data/meta_tensor.py:116: UserWarning: The given NumPy array is not writable, and PyTorch does

Above command is same with the following command line:

In [19]:
!monai-deploy exec "mednist_classifier_monaideploy.py" -i "input/AbdomenCT_007000.jpeg" -o output -m "classifier.zip"

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 410750, Operator ID: 91aa8bda-4a3b-47fd-b393-13181eb69e2c)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 410750, Operator ID: e085c688-f8d2-4896-899a-3905105e40e8)
/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:111: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead with `channel_dim='no_channel'`.
  warn_deprecated(obj, msg, warning_category)
/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/monai/data/meta_tensor.py:116: UserWarning: The given NumPy array is not writable, and PyTorch does

In [20]:
!cat output/output.json

"AbdomenCT"